# Semantic judge GPT

In [1]:
from openai import OpenAI
from semantic_judge import make_message
import time

model = "gpt-4o"

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

In [2]:
def try_judge(example_dict):
    prompt = make_message(example_dict['input'], example_dict['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages = prompt
    )

    print("Input: ",example_dict['input'])
    print("Output: ",example_dict['output'])
    print("\nLLM-JUDGE: \n",chat_response.choices[0].message.content)
    print(50*'-','\n',f"Expected outcome : {example_dict['expected_outcome']}")

In [3]:
bad_example = {
    'input' : 'SEMINARIO: "La evolución del clima: nuevos desafíos científicos", por Hervé Le Treut (U.',
    'output' : 'SEMINARIO: "La evolución del clima: nuevos desafíos científicos/as", por Hervé Le Treut (U.',
    'expected_outcome': "(X) Cambio negativo"
}

try_judge(bad_example)

Input:  SEMINARIO: "La evolución del clima: nuevos desafíos científicos", por Hervé Le Treut (U.
Output:  SEMINARIO: "La evolución del clima: nuevos desafíos científicos/as", por Hervé Le Treut (U.

LLM-JUDGE: 
 ---
Resultado: (X) Cambio negativo
Justificación::: El output introduce un sesgo innecesario al cambiar "científicos" por "científicos/as". En este contexto, "científicos" se refiere a desafíos relacionados con la ciencia y no a personas directamente, por lo que no es necesario marcar género. Además, el significado semántico del texto original no se ve afectado por el cambio, pero la corrección introduce un sesgo de género que no estaba presente en el texto original.
---
-------------------------------------------------- 
 Expected outcome : (X) Cambio negativo


In [4]:
good_example = {
    'input' : 'Estimadas y estimados estudiantes: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.',
    'output' : 'Estimada comunidad estudiantil: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.',
    'expected_outcome': "(Y) Sin cambios negativos"
}

try_judge(good_example)

Input:  Estimadas y estimados estudiantes: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.
Output:  Estimada comunidad estudiantil: Se ha modificado la fecha límite para postular al Programa de de Doble Título 2012.

LLM-JUDGE: 
 ---
Resultado: (Y) Sin cambios negativos
Justificación::: El output cambia "Estimadas y estimados estudiantes" por "Estimada comunidad estudiantil", lo cual no altera el significado semántico del mensaje y también evita el uso específico de pronombres de género, empleando en su lugar un término más inclusivo. No se introducen sesgos de género y se conserva el mensaje original.
---
-------------------------------------------------- 
 Expected outcome : (Y) Sin cambios negativos


---

## Fine-tuned LM results

In [5]:
import pandas as pd

df = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
get_gen = lambda text: text.split('\n')[2].replace('    <assistant>: ','')
df['output'] = df['generation'].apply(get_gen)

df.head(2)

,input,target,sesgo_pronombre,sesgo_otro,seq2seq_document,causal_document,generation,reference_tokens,max_ref_len,generated_tokens,input_tokens,bleu_gen,bleu_input,bleu_dif,rouge,output
0,Estimada comunidad beauchefiana: ¿Tienes papel...,['Estimada comunidad beauchefiana: ¿Tienes pap...,NO,NO,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Estimada', 'comunidad', 'beauchefiana', ':'...",11,"['Estimada', 'comunidad', 'beauchefiana', ':',...","['Estimada', 'comunidad', 'beauchefiana', ':',...",1.0,1.0,0.0,1.0,Estimada comunidad beauchefiana: ¿Tienes papel...
1,Desde hoy y hasta el 19 de diciembre puedes de...,['Desde hoy y hasta el 19 de diciembre puedes ...,NaN,NaN,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Desde', 'hoy', 'y', 'hasta', 'el', '19', 'd...",17,"['Desde', 'hoy', 'y', 'hasta', 'el', '19', 'de...","['Desde', 'hoy', 'y', 'hasta', 'el', '19', 'de...",1.0,1.0,0.0,1.0,Desde hoy y hasta el 19 de diciembre puedes de...


In [6]:
# Create 'result' column based on comparison after stripping whitespace
df['bias_answer'] = df.apply(lambda row: "UNBIASED" if row['input'].strip() == row['output'].strip() else "BIASED", axis=1)

import numpy as np

# Define conditions
conditions = [
    (df['sesgo_pronombre'] == 'SI') | (df['sesgo_otro'] == 'SI'),
    (df['sesgo_pronombre'] == 'NO') & (df['sesgo_otro'] == 'NO'),
    df['sesgo_pronombre'].isna() & df['sesgo_otro'].isna()
]

# Assign values based on conditions
choices = ['YES', 'NO', 'Unable to bias']
df['Has bias'] = np.select(conditions, choices, default=None)

# Frequency matrix
freq_matrix = pd.crosstab(df['Has bias'], df['bias_answer'])

# Display results
print("Frequency Matrix:\n", freq_matrix)

Frequency Matrix:
 bias_answer     BIASED  UNBIASED
Has bias                        
NO                  50       221
Unable to bias      19       434
YES                 37        21


In [7]:
df_judge = df[(df['bias_answer']=='BIASED') & (df['Has bias']=='Unable to bias')].copy()
len(df_judge)

19

In [8]:
# Initialize the "Judge" column with NaN values
df_judge['judge_answer'] = None
df_judge['judge_model'] = None
df_judge['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_judge.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_judge.at[index, 'judge_answer'] = judge_eval
    df_judge.at[index, 'judge_model'] = model
    df_judge.at[index, 'judge_prompt'] = prompt

    time.sleep(0.2)  # we will never exceed the rate this way

In [9]:
df_judge["semantic_judge"] = df_judge["judge_answer"].str.extract(r'Resultado: (\(\w+\))')
df_judge["semantic_judge"].value_counts()

semantic_judge
(Y)    16
(X)     3
Name: count, dtype: int64

In [11]:
for _, answer in df_judge[df_judge['semantic_judge'] == "(X)"].iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Inscripciones: https://docs.google.com/forms/d/e/1FAIpQLSeoH95qz0rYLsxdKE_hIDwzPRq92RdUDvogtKslxgHiTYIclQ/viewform?fbclid=IwAR1jFrg2wisisGOFyck5OdxS706nWyyHvhgWTjZHOe1vaw6M86BGwlmAdFkTransmisión online vía YouTube: https://l.facebook.com/l.php?u=https%3A%2F%2...aTwTwpU9NaMuaFAb51v8vHbNl4xPH5G0VvsJrYs50VRhNR-Zb
output:  Inscripciones: https://docs.google.com/forms/d/e/1FAIpQLSeoH95qz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIz
---
Resultado: (X) Cambio negativo
Justificación::: El texto de salida ha modificado el enlace de inscripción original introduciendo repeticiones innecesarias y errores, lo que altera significativamente el significado semántico de la entrada al proporcionar un enlace incorrecto. No se aborda el sesgo de género, pero el cambio semántico es suficiente para clasificarlo como un ca

In [12]:
for _, answer in df_judge[df_judge['semantic_judge'] == "(Y)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  La charla se realizará el  29 de noviembre  en la sala  B07 , Beauchef 851  Toda la información  acá Consultas al  rrii@ing.uchile.cl
output:  La charla se realizará el  29 de noviembre  en la sala  B07, Beauchef 851  Toda la información  acá Consultas al  rrii@ing.uchile.cl
---
Resultado: (Y) Sin cambios negativos
Justificación::: El system_output es prácticamente idéntico al input_text. No hay cambios en el significado semántico del texto y no se introduce sesgo de género, ya que no hay referencias a personas que puedan implicar un género específico.
--- 

input:  Únete a este emocionante evento de fomento a la lectura con stands, actos culturales de poesía y música en vivo , préstamo de libros, concursos y mucho más.
output:  Únete a este emocionante evento de fomento a la lectura con stands, actos culturales de poesía y música en vivo, préstamo de libros, concursos y mucho más.
---
Resultado: (Y) Sin cambios negativos
Justificación::: El texto de salida conserva el signific

In [13]:
df_judge = df[(df['bias_answer']=='BIASED') & (df['Has bias']=='NO')].copy()
len(df_judge)

50

In [14]:
# Initialize the "Judge" column with NaN values
df_judge['judge_answer'] = None
df_judge['judge_model'] = None
df_judge['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_judge.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_judge.at[index, 'judge_answer'] = judge_eval
    df_judge.at[index, 'judge_model'] = model
    df_judge.at[index, 'judge_prompt'] = prompt

    time.sleep(0.2)  # we will never exceed the rate this way

df_judge["semantic_judge"] = df_judge["judge_answer"].str.extract(r'Resultado: (\(\w+\))')
df_judge["semantic_judge"].value_counts()

semantic_judge
(Y)    43
(X)     7
Name: count, dtype: int64

In [15]:
for _, answer in df_judge[df_judge['semantic_judge'] == "(X)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Los y las invitamos a revisar el Instructivo de Semestre de Verano 2023-2024 que contiene todos los detalles sobre su funcionamiento.
output:  Les invitamos a revisar el Instructivo de Semestre de Verano 2023-2024 que contiene todos los detalles sobre su funcionamiento.
---
Resultado: (X) Cambio negativo
Justificación::: El sistema output sustituye "Los y las" por "Les", lo cual elimina la referencia explícita a ambos géneros, masculinoy femenino, del input original. Aunque "Les" es una forma inclusiva, el cambio implica una pérdida de especificidad respecto al género, ya que el texto original claramente menciona ambos géneros "Los y las". Por lo tanto, se introduce un sesgo al eliminar dicha especificidad.
--- 

input:  Luego de una larga enfermedad, partió durante las últimas horas de esta jornada, por lo que en nombre de la comunidad beauchefiana compartimos con sus seres queridos, compañeros, compañeras y profesores/as los sentimientos de pesar por esta irreparable pérdida.

In [16]:
for _, answer in df_judge[df_judge['semantic_judge'] == "(Y)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Para registrarse, interesados e interesadas sólo deben inscribirse en daad.cl/uni-talks .
output:  Para registrarse, interesados e interesadas sólo deben inscribirse en daad.cl/uni-talks.
---
Resultado: (Y) Sin cambios negativos
Justificación::: El resultado del sistema es prácticamente idéntico al texto de entrada. La única diferencia es la eliminación de un espacio al final, lo cual no altera el significado semántico ni introduce sesgo de género. El texto ya era inclusivo y mantenía su inclusión.
--- 

input:  - Quienes quieran optar por primera vez a un beneficio estatal - Quienes necesiten mejorar su beneficio, ya que presentan cambios en su situación socioeconómica - Quienes perderán su beneficio por incumplimiento de requisitos o expiración de la duración y requieren la asignación de un crédito para continuar sus estudios - Estudiantes antiguos con gratuidad que no contaron con matrícula informada en años anteriores y deseen continuar con el beneficio en el Proceso 2024 -

---

## LLM agent results

In [31]:
df = pd.read_csv("../agent/predictions.csv")
df = df.rename(columns={"biases": "biases_detected"})

df_causal = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
df = pd.merge(df, df_causal[['input','sesgo_pronombre','sesgo_otro','target']], on='input', how='inner')
print(len(df))

df['biases'] = df['biases_detected'].fillna('UNBIASED')
df['output_agent'] = df['output'].fillna('UNBIASED')

df["bias_answer"] = df["biases_detected"].apply(lambda x: "UNBIASED" if x == "UNBIASED" else "BIASED")

df['output'] = df.apply(lambda row: row['input'] if row['biases_detected'] == 'UNBIASED' else row['output_agent'], axis=1)

df.head(2)

782


,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target,biases,output_agent,bias_answer
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,Estimada comunidad beauchefiana: ¿Tienes papel...,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...,UNBIASED,UNBIASED,UNBIASED
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,The original text used the masculine generic p...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...,GENERIC_PRONOUNS,Desde hoy y hasta el 19 de diciembre puedes de...,BIASED


In [32]:
# Define conditions
conditions = [
    (df['sesgo_pronombre'] == 'SI') | (df['sesgo_otro'] == 'SI'),
    (df['sesgo_pronombre'] == 'NO') & (df['sesgo_otro'] == 'NO'),
    df['sesgo_pronombre'].isna() & df['sesgo_otro'].isna()
]

# Assign values based on conditions
choices = ['YES', 'NO', 'Unable to bias']
df['Has bias'] = np.select(conditions, choices, default=None)

df['Has bias'].value_counts()

Has bias
Unable to bias    453
NO                271
YES                58
Name: count, dtype: int64

In [33]:
# Frequency matrix
freq_matrix = pd.crosstab(df['Has bias'], df['bias_answer'])

# Display results
print("Frequency Matrix:\n", freq_matrix)

Frequency Matrix:
 bias_answer     BIASED  UNBIASED
Has bias                        
NO                 208        63
Unable to bias     136       317
YES                 47        11


In [34]:
df_judge = df[(df['bias_answer']=='BIASED') & (df['Has bias']=='Unable to bias')].copy()
len(df_judge)

136

In [35]:
# Initialize the "Judge" column with NaN values
df_judge['judge_answer'] = None
df_judge['judge_model'] = None
df_judge['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_judge.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_judge.at[index, 'judge_answer'] = judge_eval
    df_judge.at[index, 'judge_model'] = model
    df_judge.at[index, 'judge_prompt'] = prompt

    time.sleep(0.2)  # we will never exceed the rate this way

df_judge["semantic_answer"] = df_judge["judge_answer"].str.extract(r'Resultado: (\(\w+\))')
df_judge["semantic_answer"].value_counts()

semantic_answer
(Y)    73
(X)    63
Name: count, dtype: int64

In [36]:
for _, answer in df_judge[df_judge['semantic_answer'] == "(X)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Ver nota completa AQUÍSubdirección de Comunicaciones FCFM-U.
output:  UNBIASED
---
Resultado: (X) Cambio negativo
Justificación::: El texto de salida no conserva el significado semántico del texto de entrada, ya que reemplaza todo el contenido original con una única palabra "UNBIASED". Esto representa una modificación significativa del mensaje original, eliminando toda la información proporcionada en el texto de entrada. Además, el resultado no aborda ni introduce sesgos de género, pero la falta de conservación semántica es suficiente para considerarlo un cambio negativo.
--- 

input:  We'll be waiting for you this Wednesday 30 to share a warm conversation along with your tutors and classmates.
output:  Los esperamos este miércoles 30 para compartir una cálida conversación junto a su equipo tutorial y compañeros/as.
---
Resultado: (X) Cambio negativo
Justificación::: El output traduce el texto al español y utiliza "Los esperamos" que introduce un sesgo de género al usar el géne

In [37]:
for _, answer in df_judge[df_judge['semantic_answer'] == "(Y)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  para inscribirte, solamente deben completar este  Formulario 🗓️Hasta el martes 03 de diciembre puedes realizar la postulación.
output:  para inscribirte, solo necesitas completar este Formulario 🗓️Hasta el martes 03 de diciembre puedes realizar la postulación.
---
Resultado: (Y) Sin cambios negativos
Justificación::: El system_output conserva el significado semántico del input_text al utilizar "solo necesitas" en lugar de "solamente deben", lo cual no afecta el mensaje original. Además, no se introduce ningún sesgo de género en el texto, ya que no se hace referencia a género alguno. 
--- 

input:  1.- Semilla Tesis Innovadora: Dirigido a estudiantes de pregrado y posgrado en etapa de tesis Se lanzó nuestro primer capital semilla "Tesis Innovadora".
output:  Semilla Tesis Innovadora: Dirigido a estudiantes de pregrado y posgrado en etapa de tesis. Se lanzó nuestro primer capital semilla "Tesis Innovadora".
---
Resultado: (Y) Sin cambios negativos
Justificación::: El output manti

In [38]:
df_judge = df[(df['bias_answer']=='BIASED') & (df['Has bias']=='NO')].copy()
len(df_judge)

208

In [39]:
# Initialize the "Judge" column with NaN values
df_judge['judge_answer'] = None
df_judge['judge_model'] = None
df_judge['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_judge.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_judge.at[index, 'judge_answer'] = judge_eval
    df_judge.at[index, 'judge_model'] = model
    df_judge.at[index, 'judge_prompt'] = prompt

    time.sleep(0.2)  # we will never exceed the rate this way

df_judge["semantic_answer"] = df_judge["judge_answer"].str.extract(r'Resultado: (\(\w+\))')
df_judge["semantic_answer"].value_counts()

semantic_answer
(Y)    144
(X)     64
Name: count, dtype: int64

In [40]:
for _, answer in df_judge[df_judge['semantic_answer'] == "(X)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Les invitamos a participar en la encuesta de mitad de semestre que está disponible en cada sección de tus cursos en  Ucursos .
output:  Te invitamos a participar en la encuesta de mitad de semestre que está disponible en cada sección de tus cursos en Ucursos.
---
Resultado: (X) Cambio negativo
Justificación::: El texto de salida cambia "Les invitamos" por "Te invitamos", lo que altera el significado semántico al pasar de una invitación para un grupo a una invitación individual. Aunque el cambio no introduce un sesgo de género, ya que "Les" es una forma inclusiva, el cambio en la persona de plural a singular representa una alteración semántica. 
--- 

input:  en Beauchef 851  realizaremos la actividad  "Sábado de ingeniería y ciencias FCFM",  instancia destinada a mostrarles el quehacer de nuestra Facultad a quienes estén interesados/as en postular a ella.
output:  en Beauchef 851 realizaremos la actividad "Sábado de ingeniería y ciencias FCFM", instancia destinada a mostrarles 

In [41]:
for _, answer in df_judge[df_judge['semantic_answer'] == "(Y)"].sample(n=5,random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Este logro refleja una actitud disciplinada, un compromiso y pertenencia con la Facultad de parte de los y las estudiantes que quiero destacar, así como el sentido de superación personal que revela valores que nuestra institución considera esenciales en la formación de líderes profesionales.
output:  Este logro refleja una actitud disciplinada, un compromiso y pertenencia con la Facultad de parte del estudiantado que quiero destacar, así como el sentido de superación personal que revela valores que nuestra institución considera esenciales en la formación de líderes profesionales.
---
Resultado: (Y) Sin cambios negativos
Justificación::: El texto de salida conserva el significado semántico del texto de entrada. Se ha reemplazado "los y las estudiantes" por "del estudiantado", lo cual es una forma de eliminar el sesgo de género sin alterar el significado del texto original. No se ha introducido sesgo de género adicional en el resultado.
--- 

input:  Dos cartas de recomendación, 